# Attempting to make an algorithm that ranks college football teams.

### **PROCESS AND PREPARE DATA**

In [ ]:
## LOAD game data AND team data INTO DATAFRAMES ##

import pandas as pd
import numpy as np
# Load data into pandas DataFrame from "/lakehouse/default/" + "Files/CFB_Game_Data.csv"
games = pd.read_csv("/lakehouse/default/" + "Files/CFB_Game_Data.csv")
games['Date'] = pd.to_datetime(games.Date)
display(games)
# Load data into pandas DataFrame from "/lakehouse/default/" + "Files/CFB_Team_Data.csv"
teams = pd.read_csv("/lakehouse/default/Files/CFB_Team_Data.csv")
display(teams)

In [2]:
## FILTERS TO ONLY REGULAR SZN GAMES ##
games = games.loc[games['Date'] < '2023-12-03']



StatementMeta(, 1998dcad-5fd2-4213-9d65-2525a331fe4f, 4, Finished, Available)

In [ ]:
## SET INDEX OF games TO Rk
games.set_index("Rk", inplace = True)
games.head()


In [ ]:
## CHECK FOR NULLS IN games ##
games.isnull().sum() # 0 nulls


In [ ]:
## CHECK FOR NULLS IN teams ##
teams.isnull().sum() # 0 nulls

In [6]:
## ELIMINATE THE TEAMS RANKING FROM THE 'Winner' AND 'Loser' COLUMNS ##
# Code generated by Data Wrangler for pandas DataFrame #
# Replace all instances of "(n)" with "" in columns: 'Winner', 'Loser' #

def clean_data(games):
    for x in range(26):
        games['Winner'] = games['Winner'].str.replace(f"({x})", "", regex=False)
        games['Loser'] = games['Loser'].str.replace(f"({x})", "", regex=False)
    return games

games_clean = clean_data(games.copy())


StatementMeta(, 1998dcad-5fd2-4213-9d65-2525a331fe4f, 8, Finished, Available)

In [7]:
## ELIMINATE SPACES IN FRONT OF STRINGS IN 'Winner' AND 'Loser' COLUMNS ##

games_clean['Winner'] = games_clean['Winner'].str.strip()
games_clean['Loser'] = games_clean['Loser'].str.strip()



StatementMeta(, 1998dcad-5fd2-4213-9d65-2525a331fe4f, 9, Finished, Available)

In [8]:
## FIX TEAM NAMES ##

teams.loc[teams['School'] == 'LSU', 'School'] = 'Louisiana State'
teams.loc[teams['School'] == 'Pitt', 'School'] = 'Pittsburgh'
teams.loc[teams['School'] == 'UCF', 'School'] = 'Central Florida'
teams.loc[teams['School'] == 'SMU', 'School'] = 'Southern Methodist'
teams.loc[teams['School'] == 'UAB', 'School'] = 'Alabama-Birmingham'
teams.loc[teams['School'] == 'UTSA', 'School'] = 'Texas-San Antonio'
teams.loc[teams['School'] == 'BYU', 'School'] = 'Brigham Young'
teams.loc[teams['School'] == 'USC', 'School'] = 'Southern California'
teams.loc[teams['School'] == 'FIU', 'School'] = 'Florida International'
teams.loc[teams['School'] == 'JMU', 'School'] = 'James Madison'
teams.loc[teams['School'] == 'TCU', 'School'] = 'Texas Christian'
teams.loc[teams['School'] == 'UTEP', 'School'] = 'Texas-El Paso'
teams.loc[teams['School'] == 'Ole Miss', 'School'] = 'Mississippi'

StatementMeta(, 1998dcad-5fd2-4213-9d65-2525a331fe4f, 10, Finished, Available)

In [9]:
## ADD 'fcs_team_involved' COLUMN TO 'games_clean' DATAFRAME AND SET ALL VALUES TO NaN. ##

games_clean['fcs_team_involved'] = np.NaN


StatementMeta(, 1998dcad-5fd2-4213-9d65-2525a331fe4f, 11, Finished, Available)

In [ ]:
## LOAD 'FCS_Teams.csv' INTO A NEW DATAFRAME 'fcs_teams' ##

import pandas as pd
# Load data into pandas DataFrame from "/lakehouse/default/" + "Files/FCS_Teams.csv"
fcs_teams = pd.read_csv("/lakehouse/default/" + "Files/FCS_Teams.csv", encoding='cp1252')
fcs_teams.drop('Index', axis = 1, inplace = True)
fcs_teams.loc[-1] = ['Texas A&M-Commerce']
fcs_teams.index = fcs_teams.index + 1
fcs_teams.loc[-1] = ['Long Island']
fcs_teams.index = fcs_teams.index + 1
fcs_teams.loc[-1] = ['Kennesaw State']

fcs_teams.loc[fcs_teams['Team'] == 'VMI', 'Team'] = 'Virginia Military Institute'
fcs_teams.loc[fcs_teams['Team'] == 'Bethune–Cookman', 'Team'] = 'Bethune-Cookman'
fcs_teams.loc[fcs_teams['Team'] == 'Nicholls', 'Team'] = 'Nicholls State'
fcs_teams.loc[fcs_teams['Team'] == 'Arkansas–Pine Bluff', 'Team'] = 'Arkansas-Pine Bluff'
fcs_teams.loc[fcs_teams['Team'] == 'Gardner–Webb', 'Team'] = 'Gardner-Webb'
fcs_teams.loc[fcs_teams['Team'] == 'Tarleton', 'Team'] = 'Tarleton State'
fcs_teams.loc[fcs_teams['Team'] == 'Houston Christian', 'Team'] = 'Houston Baptist'
fcs_teams.loc[fcs_teams['Team'] == 'Central Connecticut', 'Team'] = 'Central Connecticut State'
fcs_teams.loc[fcs_teams['Team'] == 'Saint Francis (PA)', 'Team'] = 'St. Francis (PA)'
fcs_teams.loc[fcs_teams['Team'] == 'UC Davis', 'Team'] = 'California-Davis'
fcs_teams.loc[fcs_teams['Team'] == 'UT Martin', 'Team'] = 'Tennessee-Martin'
fcs_teams.loc[fcs_teams['Team'] == 'McNeese', 'Team'] = 'McNeese State'
fcs_teams.loc[fcs_teams['Team'] == 'The Citadel', 'Team'] = 'Citadel'


display(fcs_teams)


In [11]:
## POPULATE 'fcs_team_involved' COLUMN ##

games_clean['fcs_team_involved'] = games_clean['fcs_team_involved'].fillna(False)

for team in fcs_teams['Team']:    
    for i in games_clean['Loser']:
        games_clean.loc[team == games_clean['Loser'],
            'fcs_team_involved'] = True
        games_clean.loc[team == games_clean['Winner'],
            'fcs_team_involved'] = True
        



StatementMeta(, 1998dcad-5fd2-4213-9d65-2525a331fe4f, 13, Finished, Available)

In [ ]:
## MAKES NEW DATAFRAME fbs_games THAT ELIMINATES GAMES INVOLVING AN FCS TEAM ##
## we only want to include games involving two D1 FBS teams ##
fbs_games = games_clean.loc[games_clean['fcs_team_involved']==False]
fbs_games['fcs_team_involved'].value_counts()
fbs_games.drop('fcs_team_involved', axis = 1, inplace = True)




In [13]:
## UPDATES WINS, LOSSES, AND PCT ##

gf = fbs_games.value_counts('Winner').rename_axis('Team').reset_index(name = 'Wins')
gf2 = fbs_games.value_counts('Loser').rename_axis('Team').reset_index(name = 'Losses')
gf.head()

for team in teams['School']:
   row = gf.loc[gf.Team == team]
   if row.empty:
      teams.loc[teams['School'] == team, 'Reg_W'] = 0
   else:   
      teams.loc[teams['School'] == team, 'Reg_W'] = row.iloc[0]['Wins']
      
   #Some teams dont show up in losses df
   row2 = gf2.loc[gf2.Team == team]
   if row2.empty:
      teams.loc[teams['School'] == team, 'Reg_L'] = 0
   else:
      teams.loc[teams['School'] == team, 'Reg_L'] = row2.iloc[0]['Losses']

teams['Reg_Pct'] = (teams['Reg_W']) / (teams['Reg_W'] + teams['Reg_L'])
teams = teams.round(3)

StatementMeta(, 1998dcad-5fd2-4213-9d65-2525a331fe4f, 15, Finished, Available)

In [ ]:
## ADD 'score_diff' COLUMN TO fbs_games DATAFRAME ##
fbs_games['score_diff'] = fbs_games['Win_Pts'] - fbs_games['Lose_Pts']
fbs_games.head()

### **ANALYZE DATA TO MAKE A RANKING ALGORITHM**

In [ ]:
## RANKING ALGORITHM THAT ONLY TAKES INTO ACCOUNT WINS/LOSSES AND STRENGTH OF SCHEDULE ##
## ONLY USES GAMES FROM THE REGULAR SEASON, WHERE BOTH TEAMS INVOLVED WERE FBS TEAMS ##
## jt_rating = (SUM of [opponent win pcts. + 0.1] for each game won) - (SUM of [1.1 - opponent win pcts.] for each game lost) ##

Team_Rankings = teams.copy()
Team_Rankings['jt_rating'] = 0

for team in Team_Rankings.School:
    df = fbs_games.loc[team == fbs_games['Winner']]
    df2 = fbs_games.loc[team == fbs_games['Loser']]
    for loser in df['Loser']:
        row = Team_Rankings.loc[Team_Rankings['School'] == loser]
        lpct = row.iloc[0]['Reg_Pct']
        df.loc[df['Loser'] == loser, 'lpct'] = lpct
        df['pts'] = (0.1 + df['lpct']) 
        Team_Rankings.loc[Team_Rankings['School'] == team, 'jt_rating'] = df['pts'].sum()
    for winner in df2['Winner']:
        row2 = Team_Rankings.loc[Team_Rankings['School'] == winner]
        wpct = row2.iloc[0]['Reg_Pct']
        df2.loc[df2['Winner'] == winner, 'wpct'] = wpct
        df2['pts'] = ((1.1 - df2['wpct']))
        Team_Rankings.loc[Team_Rankings['School'] == team, 'jt_rating'] -= df2['pts'].sum()

Team_Rankings.sort_values('jt_rating', ascending = False).head(25)


In [ ]:
## FIRST ATTEMPT AT ALGORITHM ##
## ATTEMPTS TO USE SCORE DIFFERENTIAL, WHICH OVERVALUES 
## BLOWOUT WINS AGAINST BAD TEAMS AND UNDERVALUES CLOSE WINS AGAINTS GOOD TEAMS ##
teams['jt_rating'] = 0

for team in teams.School:
    df = fbs_games.loc[team == fbs_games['Winner']]
    df2 = fbs_games.loc[team == fbs_games['Loser']]
    for loser in df['Loser']:
        row = teams.loc[teams['School'] == loser]
        lpct = row.iloc[0]['Pct']
        df.loc[df['Loser'] == loser, 'lpct'] = lpct
        df['pts'] = (df['score_diff']) * (df['lpct']) 
        teams.loc[teams['School'] == team, 'jt_rating'] = df['pts'].sum()
    for winner in df2['Winner']:
        row2 = teams.loc[teams['School'] == winner]
        wpct = row2.iloc[0]['Pct']
        df2.loc[df2['Winner'] == winner, 'wpct'] = wpct
        df2['pts'] = (df2['score_diff']) * ((1 - df2['wpct']))
        teams.loc[teams['School'] == team, 'jt_rating'] -= df2['pts'].sum()
    

